In [1]:
import pandas as pd
import skimage.io as io
import matplotlib.pyplot as plt
import cv2
import gc
import numpy as np
import skimage
import skimage.feature as feature
import glob
import os
from skimage.color import rgb2gray
from PIL import Image, ImageChops

In [ ]:
annotations1= json.load(open('D:/Harish/code/DTS/via_region_data.json'))
annotations = list(annotations1.values())
annotations = [a for a in annotations if a['regions']]
len(annotations)

In [ ]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [16]:
def extract_bboxes(mask):
    """Compute bounding boxes from masks.
    mask: [height, width, num_instances]. Mask pixels are either 1 or 0.

    Returns: bbox array [num_instances, (y1, x1, y2, x2)].
    """
    boxes = np.zeros([mask.shape[-1], 4], dtype=np.int32)
    for i in range(mask.shape[-1]):
        m = mask[:, :, i]
        # Bounding box.
        horizontal_indicies = np.where(np.any(m, axis=0))[0]
        vertical_indicies = np.where(np.any(m, axis=1))[0]
        if horizontal_indicies.shape[0]:
            x1, x2 = horizontal_indicies[[0, -1]]
            y1, y2 = vertical_indicies[[0, -1]]
            # x2 and y2 should not be part of the box. Increment by 1.
            x2 += 1
            y2 += 1
        else:
            # No mask for this instance. Might happen due to
            # resizing or cropping. Set bbox to zeros
            x1, x2, y1, y2 = 0, 0, 0, 0
        #for keras tensorflow
        #boxes[i] = np.array([y1, x1, y2, x2]) 
        #for pytorch
        boxes[i] = np.array([x1, y1, x2, y2])
    return boxes.astype(np.int32)

In [3]:
path = 'D:/Harish/code/DTS/dts_data'
out = 'D:/Harish/code/DTS/new_classification/'
ignore = []
complete = []
image_info = {}
filename = []
labels = []
ignore_ = []
for a in annotations:
    polygons = [r['shape_attributes'] for r in a['regions'].values()]
    values = [r['region_attributes'] for r in a['regions'].values() if r['region_attributes']]
    if len(values)>0:
        if 'class' in values[0].keys():
            classes = [r['class'].strip().lower() for r in values if r['class'].strip().lower()!='']
        elif 'Class' in values[0].keys():
            classes = [r['Class'].strip().lower() for r in values if r['Class'].strip().lower()!='']
        #elif 'color' in values[0].keys():
        #    continue
    for com_ in com[1:]:
        classes = [r.replace(com_.lower(),'') for r in classes]
    classes = [r.replace('_','') for r in classes]
    classes = [r.replace(' ','') for r in classes]
    classes = [r.replace(',','') for r in classes]
    classes = [r.replace(';','') for r in classes]
    classes = [r.replace('poor','') for r in classes]
    classes = [r.replace('obstructedbybox','') for r in classes]
    #classes = ['streetname' for r in classes if 'streetname' in r]
    #classes = ['trafficlight' for r in classes if 'trafficlight' in r]
    if a['filename'] in ignore:
        continue
    image_path = os.path.join(path, a['filename'])
    if not os.path.exists(image_path):
        print('here')
        ignore.append(a['filename'])
        print(a['filename'])
        continue
        
    image = skimage.io.imread(image_path)
    height, width = image.shape[:2]
    #print(a['filename'])
    if len(polygons) != len(classes):
        print(a['filename'])
        continue
    for p in range(0,len(polygons)):
        if polygons[p]['name'] == 'polygon':
            po = polygons[p]
            mask = np.zeros([height, width, 1],dtype=np.uint8)
            rr, cc = skimage.draw.polygon(po['all_points_y'], po['all_points_x'])
            mask[rr, cc, 0] = 1
            x1,y1,x2,y2 = extract_bboxes(mask)[0]
            
        if polygons[p]['name'] == 'rect':
            po = polygons[p]
            x1,y1,x2,y2 = po['x'],po['y'],po['x']+po['width'],po['y']+po['height']
            
        if classes[p].lower() not in ignore_:
            image_info = {}
            ann = {}
            image_info['filename'] = str(p)+a['filename']
            #print(bbox[p])
            #x1, y1, x2, y2 = bbox[p]
            image_info['width'] = x2-x1
            image_info['height'] = y2-y1
            ann['labels'] =  classes[p]
            #ann['labels'] =  np.array([class_dict[classes[p].lower()]], dtype=np.int32)
            image_info['ann'] =ann
            crop_img = image[y1:y2, x1:x2].copy()
            print(crop_img.shape)
            if 0 in crop_img.shape:
                print('here2')
                print(a['filename'])
                continue
            crop_img = cv2.cvtColor(crop_img,cv2.COLOR_BGR2RGB)
            cv2.imwrite(out+image_info['filename'] ,crop_img)
            complete.append(image_info)

poles335_predicted.jpg,960,39,986,83,0

poles335_predicted.jpg,695,0,716,33,0

poles335_predicted.jpg,114,302,148,347,1

poles335_predicted.jpg,485,804,508,837,1

poles606_predicted.jpg,157,15,183,60,0

poles606_predicted.jpg,811,663,829,697,0

poles606_predicted.jpg,159,21,183,60,1

poles154_predicted.jpg,893,411,934,465,0

poles154_predicted.jpg,721,183,757,233,0

poles154_predicted.jpg,928,819,956,867,1

poles154_predicted.jpg,766,611,801,658,1

poles1078_predicted.jpg,142,35,171,77,0

poles966_predicted.jpg,917,520,945,552,0

poles966_predicted.jpg,291,212,323,255,0

poles1471_predicted.jpg,982,55,1017,102,0

poles1471_predicted.jpg,543,751,568,795,0

poles1471_predicted.jpg,729,504,756,552,1

poles1471_predicted.jpg,447,756,481,797,1

poles1471_predicted.jpg,796,443,820,493,1

poles1471_predicted.jpg,614,615,643,654,1

poles1471_predicted.jpg,200,646,232,695,1

poles1471_predicted.jpg,338,802,369,832,1

poles1471_predicted.jpg,957,225,982,265,1

poles382_predicted.jpg,802,423,828,

poles794_predicted.jpg,505,655,547,702,0

poles794_predicted.jpg,521,645,553,685,0

poles794_predicted.jpg,151,252,195,309,0

poles794_predicted.jpg,402,110,445,139,0

poles1017_predicted.jpg,235,0,280,102,0

poles445_predicted.jpg,636,441,669,469,0

poles61_predicted.jpg,401,35,431,92,0

poles61_predicted.jpg,620,522,653,553,0

poles61_predicted.jpg,122,96,154,130,0

poles61_predicted.jpg,221,165,258,211,0

poles458_predicted.jpg,22,172,56,213,0

poles458_predicted.jpg,193,352,222,394,0

poles458_predicted.jpg,911,582,935,608,0

poles458_predicted.jpg,914,582,933,605,1

poles1057_predicted.jpg,852,262,891,379,0

poles1057_predicted.jpg,234,126,281,258,0

poles1651_predicted.jpg,1115,769,1142,817,0

poles1651_predicted.jpg,92,43,128,89,0

poles1651_predicted.jpg,959,578,994,624,0

poles1651_predicted.jpg,816,405,848,443,0

poles1651_predicted.jpg,665,105,689,138,1

poles1651_predicted.jpg,562,184,589,235,1

poles1651_predicted.jpg,96,726,122,774,1

poles1527_predicted.jpg,693,108,738,1

poles1200_predicted.jpg,905,671,928,716,1

poles1200_predicted.jpg,104,506,135,539,1

poles1200_predicted.jpg,1,694,24,724,1

poles1585_predicted.jpg,675,387,706,415,0

poles1585_predicted.jpg,1124,754,1145,799,0

poles1585_predicted.jpg,359,161,389,210,0

poles1585_predicted.jpg,668,622,696,662,0

poles1585_predicted.jpg,920,192,945,216,0

poles1585_predicted.jpg,988,858,1019,901,0

poles1585_predicted.jpg,219,36,249,84,0

poles1585_predicted.jpg,670,622,695,663,1

poles1585_predicted.jpg,989,859,1019,902,1

poles993_predicted.jpg,578,614,611,663,0

poles993_predicted.jpg,966,620,1009,668,0

poles993_predicted.jpg,220,844,258,893,0

poles993_predicted.jpg,835,445,861,492,1

poles993_predicted.jpg,699,544,725,590,1

poles993_predicted.jpg,371,763,396,806,1

poles993_predicted.jpg,1045,313,1073,356,1

poles1193_predicted.jpg,835,179,865,217,0

poles1193_predicted.jpg,551,540,578,593,0

poles1193_predicted.jpg,15,793,52,836,0

poles1193_predicted.jpg,820,466,848,502,1

poles1193_predicte

In [ ]:
train = complete[:200]
val = complete[200:]
len(val)

In [ ]:
with open('train_ele.json', 'w') as json_file:  
    json.dump(train, json_file, cls=NumpyEncoder)
with open('val_ele.json', 'w') as json_file:  
    json.dump(val, json_file, cls=NumpyEncoder)